In [ ]:
#|default_exp core

# Cosette's source

## Setup

In [ ]:
#| export
from fastcore import imghdr
from fastcore.utils import *
from fastcore.meta import delegates

import inspect, typing, mimetypes, base64, json, ast
from collections import abc
from random import choices
from string import ascii_letters,digits

from msglm import mk_msg_openai as mk_msg, mk_msgs_openai as mk_msgs

from openai import types
from openai import Completion,OpenAI,NOT_GIVEN,AzureOpenAI
from openai.resources import chat
from openai.resources.chat import Completions
from openai.types.chat.chat_completion import ChatCompletion, ChatCompletionMessage
from openai.types.completion_usage import CompletionUsage

from toolslm.funccall import *

try: from IPython import display
except: display=None

In [ ]:
#| hide
from nbdev import show_doc

In [ ]:
#| export
_all_ = ['mk_msg', 'mk_msgs']

In [ ]:
#| export
empty = inspect.Parameter.empty

In [ ]:
#| hide
from datetime import datetime
from pprint import pprint

def print_columns(items, cols=3, width=30):
    for i in range(0, len(items), cols):
        row = items[i:i+cols]
        print(''.join(item[:width-1].ljust(width) for item in row))


client = OpenAI()
models = client.models.list()
print(f"Available models as of {datetime.now().strftime('%Y-%m-%d')}:\n")
print_columns(sorted([m.id for m in models]))

Available models as of 2025-01-31:

babbage-002                   chatgpt-4o-latest             dall-e-2                      
dall-e-3                      davinci-002                   ft:gpt-4o-2024-08-06:answerai 
ft:gpt-4o-2024-08-06:answerai ft:gpt-4o-2024-08-06:answerai ft:gpt-4o-mini-2024-07-18:ans 
ft:gpt-4o-mini-2024-07-18:ans gpt-3.5-turbo                 gpt-3.5-turbo-0125            
gpt-3.5-turbo-1106            gpt-3.5-turbo-16k             gpt-3.5-turbo-instruct        
gpt-3.5-turbo-instruct-0914   gpt-4                         gpt-4-0125-preview            
gpt-4-1106-preview            gpt-4-turbo                   gpt-4-turbo-2024-04-09        
gpt-4-turbo-preview           gpt-4o                        gpt-4o-2024-05-13             
gpt-4o-2024-08-06             gpt-4o-2024-11-20             gpt-4o-audio-preview          
gpt-4o-audio-preview-2024-10- gpt-4o-audio-preview-2024-12- gpt-4o-mini                   
gpt-4o-mini-2024-07-18        gpt-4o-mini-audio-previe

*NB* Since index into models is often hardcoded in consuming code, *always append newer entries to the end of the list* to avoid breaking code that consumes this library.

In [ ]:
#| exports
models = 'o1-preview', 'o1-mini', 'gpt-4o', 'gpt-4o-mini', 'gpt-4-turbo', 'gpt-4', 'gpt-4-32k', 'gpt-3.5-turbo', 'gpt-3.5-turbo-instruct', 'o1', 'o3-mini', 'chatgpt-4o-latest'

`o1` should support images while `o1-preview`, `o1-mini`, `o3-mini` do not support images.

In [ ]:
#| exports
text_only_models = 'o1-preview', 'o1-mini', 'o3-mini'

In [ ]:
#| exports
has_streaming_models = set(models) - set(('o1', 'o1-mini', 'o3-mini'))
has_system_prompt_models = set(models) - set(('o1-mini', 'o3-mini'))
has_temperature_models = set(models) - set(('o1', 'o1-mini', 'o3-mini'))

In [ ]:
model = models[2]
model

'gpt-4o'

For examples, we'll use GPT-4o.

## OpenAI SDK

In [ ]:
cli = OpenAI().chat.completions

In [ ]:
m = {'role': 'user', 'content': "I'm Jeremy"}
r = cli.create(messages=[m], model=model, max_completion_tokens=100)
r

Hello, Jeremy! How can I assist you today?

<details>

- id: chatcmpl-AvrnGqI9TV3oTJuCLAL1ThXpqJptc
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello, Jeremy! How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))]
- created: 1738354842
- model: gpt-4o-2024-08-06
- object: chat.completion
- service_tier: default
- system_fingerprint: fp_4691090a87
- usage: CompletionUsage(completion_tokens=12, prompt_tokens=9, total_tokens=21, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
- _request_id: req_10a1b2a0e683163dfe746c6caecf97c9
- __exclude_fields__: {'__exclude_fields__', '_request_id'}

</details>

### Formatting output

In [ ]:
#| exports
def find_block(r:abc.Mapping, # The message to look in
              ):
    "Find the message in `r`."
    m = nested_idx(r, 'choices', 0)
    if not m: return m
    if hasattr(m, 'message'): return m.message
    return m.delta

In [ ]:
#| exports
def contents(r):
    "Helper to get the contents from response `r`."
    blk = find_block(r)
    if not blk: return r
    if hasattr(blk, 'content'): return getattr(blk,'content')
    return blk

In [ ]:
contents(r)

'Hello, Jeremy! How can I assist you today?'

In [ ]:
#| exports
@patch
def _repr_markdown_(self:ChatCompletion):
    det = '\n- '.join(f'{k}: {v}' for k,v in dict(self).items())
    res = contents(self)
    if not res: return f"- {det}"
    return f"""{contents(self)}

<details>

- {det}

</details>"""

In [ ]:
r

Hello, Jeremy! How can I assist you today?

<details>

- id: chatcmpl-AvrnGqI9TV3oTJuCLAL1ThXpqJptc
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello, Jeremy! How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))]
- created: 1738354842
- model: gpt-4o-2024-08-06
- object: chat.completion
- service_tier: default
- system_fingerprint: fp_4691090a87
- usage: CompletionUsage(completion_tokens=12, prompt_tokens=9, total_tokens=21, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
- _request_id: req_10a1b2a0e683163dfe746c6caecf97c9
- __exclude_fields__: {'__exclude_fields__', '_request_id'}

</details>

In [ ]:
r.usage

In: 9; Out: 12; Total: 21

In [ ]:
#| exports
def usage(inp=0, # Number of prompt tokens
          out=0  # Number of completion tokens
         ):
    "Slightly more concise version of `CompletionUsage`."
    return CompletionUsage(prompt_tokens=inp, completion_tokens=out, total_tokens=inp+out)

In [ ]:
usage(5)

In: 5; Out: 0; Total: 5

In [ ]:
#| exports
@patch
def __repr__(self:CompletionUsage): return f'In: {self.prompt_tokens}; Out: {self.completion_tokens}; Total: {self.total_tokens}'

In [ ]:
r.usage

In: 9; Out: 12; Total: 21

In [ ]:
#| exports
@patch
def __add__(self:CompletionUsage, b):
    "Add together each of `input_tokens` and `output_tokens`"
    return usage(self.prompt_tokens+b.prompt_tokens, self.completion_tokens+b.completion_tokens)

In [ ]:
r.usage+r.usage

In: 18; Out: 24; Total: 42

In [ ]:
#| export
def wrap_latex(text, md=True):
    "Replace OpenAI LaTeX codes with markdown-compatible ones"
    text = re.sub(r"\\\((.*?)\\\)", lambda o: f"${o.group(1)}$", text)
    res = re.sub(r"\\\[(.*?)\\\]", lambda o: f"$${o.group(1)}$$", text, flags=re.DOTALL)
    if md: res = display.Markdown(res)
    return res

### Creating messages

Creating correctly formatted `dict`s from scratch every time isn't very handy, so we'll import a couple of helper functions from the `msglm` library.

Let's use `mk_msg` to recreate our msg `{'role': 'user', 'content': "I'm Jeremy"}` from earlier.

In [ ]:
prompt = "I'm Jeremy"
m = mk_msg(prompt)
r = cli.create(messages=[m], model=model, max_completion_tokens=100)
r

Hi Jeremy! How can I assist you today?

<details>

- id: chatcmpl-AvrnIZ76kdH1gVP0KPGlpqaSPKzDS
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hi Jeremy! How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))]
- created: 1738354844
- model: gpt-4o-2024-08-06
- object: chat.completion
- service_tier: default
- system_fingerprint: fp_4691090a87
- usage: CompletionUsage(completion_tokens=11, prompt_tokens=9, total_tokens=20, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
- _request_id: req_671e207121e113e9adb369c57ca55d7c
- __exclude_fields__: {'__exclude_fields__', '_request_id'}

</details>

We can pass more than just text messages to OpenAI. As we'll see later we can also pass images, SDK objects, etc. To handle these different data types we need to pass the type along with our content to OpenAI. 

Here's an example of a multimodal message containing text and images. 

```json
{
    'role': 'user', 
    'content': [
        {'type': 'text', 'text': 'What is in the image?'},
        {'type': 'image_url', 'image_url': {'url': f'data:{MEDIA_TYPE};base64,{IMG}'}}
    ]
}
```

`mk_msg` infers the type automatically and creates the appropriate data structure. 

LLMs, don't actually have state, but instead dialogs are created by passing back all previous prompts and responses every time. With OpenAI, they always alternate *user* and *assistant*. We'll use `mk_msgs` from `msglm` to make it easier to build up these dialog lists.

In [ ]:
msgs = mk_msgs([prompt, r, "I forgot my name. Can you remind me please?"]) 
msgs

[{'role': 'user', 'content': "I'm Jeremy"},
 ChatCompletionMessage(content='Hi Jeremy! How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None),
 {'role': 'user', 'content': 'I forgot my name. Can you remind me please?'}]

In [ ]:
cli.create(messages=msgs, model=model, max_completion_tokens=200)

Your name is Jeremy. How can I help you further?

<details>

- id: chatcmpl-AvrnJUxXCbN2g1vGiOGJgmSy4Pa4G
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Your name is Jeremy. How can I help you further?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))]
- created: 1738354845
- model: gpt-4o-2024-08-06
- object: chat.completion
- service_tier: default
- system_fingerprint: fp_4691090a87
- usage: CompletionUsage(completion_tokens=13, prompt_tokens=38, total_tokens=51, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
- _request_id: req_2ff97a96015e781b8f0f116bd60defb7
- __exclude_fields__: {'__exclude_fields__', '_request_id'}

</details>

## Client

In [ ]:
#| exports
class Client:
    def __init__(self, model, cli=None):
        "Basic LLM messages client."
        self.model,self.use = model,usage(0,0)
        self.text_only = model in text_only_models
        self.c = (cli or OpenAI()).chat.completions

In [ ]:
c = Client(model)
c.use

In: 0; Out: 0; Total: 0

In [ ]:
#| exports
@patch
def _r(self:Client, r:ChatCompletion):
    "Store the result of the message and accrue total usage."
    self.result = r
    if getattr(r,'usage',None): self.use += r.usage
    return r

In [ ]:
c._r(r)
c.use

In: 9; Out: 11; Total: 20

In [ ]:
#| export
def get_stream(r):
    for o in r:
        o = contents(o)
        if o and isinstance(o, str): yield(o)

In [ ]:
#| exports
@patch
@delegates(Completions.create)
def __call__(self:Client,
             msgs:list, # List of messages in the dialog
             sp:str='', # System prompt
             maxtok=4096, # Maximum tokens
             stream:bool=False, # Stream response?
             **kwargs):
    "Make a call to LLM."
    if 'tools' in kwargs: assert not self.text_only, "Tool use is not supported by the current model type."
    if any(c['type'] == 'image_url' for msg in msgs if isinstance(msg, dict) and isinstance(msg.get('content'), list) for c in msg['content']): assert not self.text_only, "Images are not supported by the current model type."
    if stream: kwargs['stream_options'] = {"include_usage": True}
    if self.model in has_system_prompt_models:
        msgs = [mk_msg(sp, 'system')] + list(msgs)

    r = self.c.create(
        model=self.model, messages=msgs, max_completion_tokens=maxtok, stream=stream, **kwargs)
    if not stream: return self._r(r)
    else: return get_stream(map(self._r, r))

In [ ]:
msgs = [mk_msg('Hi')]

In [ ]:
c(msgs)

Hello! How can I assist you today?

<details>

- id: chatcmpl-AvrnLik2SaNaHQM1IXrStnVFXWVEI
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))]
- created: 1738354847
- model: gpt-4o-2024-08-06
- object: chat.completion
- service_tier: default
- system_fingerprint: fp_50cad350e4
- usage: CompletionUsage(completion_tokens=10, prompt_tokens=8, total_tokens=18, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
- _request_id: req_f4302a778667c07f4044baecfb5f2a1c
- __exclude_fields__: {'__exclude_fields__', '_request_id'}

</details>

In [ ]:
c.use

In: 17; Out: 21; Total: 38

In [ ]:
for o in c(msgs, stream=True): print(o, end='')

Hello! How can I assist you today?

In [ ]:
c.use

In: 25; Out: 31; Total: 56

## Tool use

In [ ]:
def sums(
    a:int,  # First thing to sum
    b:int # Second thing to sum
) -> int: # The sum of the inputs
    "Adds a + b."
    print(f"Finding the sum of {a} and {b}")
    return a + b

In [ ]:
#| export
def mk_openai_func(f): 
    sc = get_schema(f, 'parameters')
    sc['parameters'].pop('title', None)
    return dict(type='function', function=sc)

In [ ]:
#| export
def mk_tool_choice(f): return dict(type='function', function={'name':f})

In [ ]:
sysp = "You are a helpful assistant. When using tools, be sure to pass all required parameters, at minimum."

In [ ]:
a,b = 604542,6458932
pr = f"What is {a}+{b}?"
tools=[mk_openai_func(sums)]
tool_choice=mk_tool_choice("sums")

In [ ]:
msgs = [mk_msg(pr)]
r = c(msgs, sp=sysp, tools=tools)
r

- id: chatcmpl-AvrnNSGqHjn1SA17vdOi1nN56coHr
- choices: [Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_J1rMfaAWXvNuT5pcsuPuwVNo', function=Function(arguments='{"a":604542,"b":6458932}', name='sums'), type='function')]))]
- created: 1738354849
- model: gpt-4o-2024-08-06
- object: chat.completion
- service_tier: default
- system_fingerprint: fp_50cad350e4
- usage: CompletionUsage(completion_tokens=22, prompt_tokens=94, total_tokens=116, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
- _request_id: req_2f7d17c5b61a00698bc40f9c4cb91107
- __exclude_fields__: {'__exclude_fields__', '_request_id'}

In [ ]:
m = find_block(r)
m

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_J1rMfaAWXvNuT5pcsuPuwVNo', function=Function(arguments='{"a":604542,"b":6458932}', name='sums'), type='function')])

In [ ]:
tc = m.tool_calls
tc

[ChatCompletionMessageToolCall(id='call_J1rMfaAWXvNuT5pcsuPuwVNo', function=Function(arguments='{"a":604542,"b":6458932}', name='sums'), type='function')]

In [ ]:
func = tc[0].function
func

Function(arguments='{"a":604542,"b":6458932}', name='sums')

In [ ]:
#| exports
def call_func_openai(func:types.chat.chat_completion_message_tool_call.Function, ns:Optional[abc.Mapping]=None):
    return call_func(func.name, ast.literal_eval(func.arguments), ns)

In [ ]:
ns = mk_ns(sums)
res = call_func_openai(func, ns=ns)
res

Finding the sum of 604542 and 6458932


7063474

In [ ]:
#| exports
def mk_toolres(
    r:abc.Mapping, # Tool use request response
    ns:Optional[abc.Mapping]=None, # Namespace to search for tools
    obj:Optional=None # Class to search for tools
    ):
    "Create a `tool_result` message from response `r`."
    r = mk_msg(r)
    tcs = getattr(r, 'tool_calls', [])
    res = [r]
    if ns is None: ns = globals()
    if obj is not None: ns = mk_ns(obj)
    for tc in (tcs or []):
        func = tc.function
        cts = str(call_func_openai(func, ns=ns))
        res.append(mk_msg(str(cts), 'tool', tool_call_id=tc.id, name=func.name))
    return res

In [ ]:
tr = mk_toolres(r, ns=ns)
tr

Finding the sum of 604542 and 6458932


[ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_J1rMfaAWXvNuT5pcsuPuwVNo', function=Function(arguments='{"a":604542,"b":6458932}', name='sums'), type='function')]),
 {'role': 'tool',
  'content': '7063474',
  'tool_call_id': 'call_J1rMfaAWXvNuT5pcsuPuwVNo',
  'name': 'sums'}]

In [ ]:
msgs += tr

In [ ]:
res = c(msgs, sp=sysp, tools=tools)
res

The sum of 604542 and 6458932 is 7063474.

<details>

- id: chatcmpl-AvrnPGK3mzUlVD2xh6FKquLbpx7cG
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The sum of 604542 and 6458932 is 7063474.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))]
- created: 1738354851
- model: gpt-4o-2024-08-06
- object: chat.completion
- service_tier: default
- system_fingerprint: fp_50cad350e4
- usage: CompletionUsage(completion_tokens=19, prompt_tokens=126, total_tokens=145, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
- _request_id: req_c5bf38bf41fa485320cd6c2ac1db9491
- __exclude_fields__: {'__exclude_fields__', '_request_id'}

</details>

In [ ]:
class Dummy:
    def sums(
        self,
        a:int,  # First thing to sum
        b:int=1 # Second thing to sum
    ) -> int: # The sum of the inputs
        "Adds a + b."
        print(f"Finding the sum of {a} and {b}")
        return a + b

In [ ]:
tools = [mk_openai_func(Dummy.sums)]

o = Dummy()
msgs = mk_toolres("I'm Jeremy")
r = c(msgs, sp=sysp, tools=tools)
msgs += mk_toolres(r, obj=o)
res = c(msgs, sp=sysp, tools=tools)
res

Hello Jeremy! How can I assist you today?

<details>

- id: chatcmpl-AvrnQAnJhaoLpWrn4GpryGwO1awrD
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello Jeremy! How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))]
- created: 1738354852
- model: gpt-4o-2024-08-06
- object: chat.completion
- service_tier: default
- system_fingerprint: fp_50cad350e4
- usage: CompletionUsage(completion_tokens=12, prompt_tokens=106, total_tokens=118, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
- _request_id: req_0bb1f5835ad93ec9a36f7fb458238cca
- __exclude_fields__: {'__exclude_fields__', '_request_id'}

</details>

In [ ]:
msgs

[{'role': 'user', 'content': "I'm Jeremy"},
 ChatCompletionMessage(content='Hello Jeremy! How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)]

In [ ]:
tools = [mk_openai_func(Dummy.sums)]

o = Dummy()
msgs = mk_toolres(pr)
r = c(msgs, sp=sysp, tools=tools)
msgs += mk_toolres(r, obj=o)
res = c(msgs, sp=sysp, tools=tools)
res

Finding the sum of 604542 and 6458932


The sum of 604,542 and 6,458,932 is 7,063,474.

<details>

- id: chatcmpl-AvrnScUsY8LTk9Iib4DckwqXPq0EM
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The sum of 604,542 and 6,458,932 is 7,063,474.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))]
- created: 1738354854
- model: gpt-4o-2024-08-06
- object: chat.completion
- service_tier: default
- system_fingerprint: fp_50cad350e4
- usage: CompletionUsage(completion_tokens=24, prompt_tokens=132, total_tokens=156, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
- _request_id: req_0f6083e22a500b5c90c6571ae31991bb
- __exclude_fields__: {'__exclude_fields__', '_request_id'}

</details>

In [ ]:
#| exports

def _mock_id(): return 'call_' + ''.join(choices(ascii_letters+digits, k=24))

def mock_tooluse(name:str, # The name of the called function
                 res,  # The result of calling the function
                 **kwargs): # The arguments to the function
    ""
    id = _mock_id()
    func = dict(arguments=json.dumps(kwargs), name=name)
    tc = dict(id=id, function=func, type='function')
    req = dict(content=None, role='assistant', tool_calls=[tc])
    resp = mk_msg('' if res is None else str(res), 'tool', tool_call_id=id, name=name)
    return [req,resp]

This function mocks the messages needed to implement tool use, for situations where you want to insert tool use messages into a dialog without actually calling into the model.

In [ ]:
tu = mock_tooluse(name='sums', res=7063474, a=604542, b=6458932)
r = c([mk_msg(pr)]+tu, tools=tools)
r

The sum of 604542 and 6458932 is 7063474.

<details>

- id: chatcmpl-AvrnUvxuwOdBQNSuSX0z168YaraXz
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The sum of 604542 and 6458932 is 7063474.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))]
- created: 1738354856
- model: gpt-4o-2024-08-06
- object: chat.completion
- service_tier: default
- system_fingerprint: fp_50cad350e4
- usage: CompletionUsage(completion_tokens=19, prompt_tokens=111, total_tokens=130, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
- _request_id: req_b043700b108987819f58c52aa8dcaeb4
- __exclude_fields__: {'__exclude_fields__', '_request_id'}

</details>

Structured outputs

In [ ]:
#| exports
@patch
@delegates(Client.__call__)
def structured(self:Client,
               msgs: list, # Prompt
               tools:Optional[list]=None, # List of tools to make available to OpenAI model
               obj:Optional=None, # Class to search for tools
               ns:Optional[abc.Mapping]=None, # Namespace to search for tools
               **kwargs):
    "Return the value of all tool calls (generally used for structured outputs)"
    tools = listify(tools)
    if ns is None: ns=mk_ns(*tools)
    tools = [mk_openai_func(o) for o in tools]
    if obj is not None: ns = mk_ns(obj)
    res = self(msgs, tools=tools, tool_choice='required', **kwargs)
    cts = getattr(res, 'choices', [])
    tcs = [call_func_openai(t.function, ns=ns) for o in cts for t in (o.message.tool_calls or [])]
    return tcs

OpenAI's API doesn't natively support response formats, so we introduce a `structured` method to handle tool calling for this purpose. In this setup, the tool's result is sent directly to the user without being passed back to the model.

In [ ]:
c.structured(mk_msgs(pr), tools=[sums])

Finding the sum of 604542 and 6458932


[7063474]

## Chat

In [ ]:
#| exports
class Chat:
    def __init__(self,
                 model:Optional[str]=None, # Model to use (leave empty if passing `cli`)
                 cli:Optional[Client]=None, # Client to use (leave empty if passing `model`)
                 sp='', # Optional system prompt
                 tools:Optional[list]=None,  # List of tools to make available
                 tool_choice:Optional[str]=None): # Forced tool choice
        "OpenAI chat client."
        assert model or cli
        self.c = (cli or Client(model))
        self.h,self.sp,self.tools,self.tool_choice = [],sp,tools,tool_choice
    
    @property
    def use(self): return self.c.use

In [ ]:
sp = "Never mention what tools you use."
chat = Chat(model, sp=sp)
chat.c.use, chat.h

(In: 0; Out: 0; Total: 0, [])

In [ ]:
#| exports
@patch
@delegates(Completions.create)
def __call__(self:Chat,
             pr=None,  # Prompt / message
             stream:bool=False, # Stream response?
             **kwargs):
    "Add prompt `pr` to dialog and get a response"
    if isinstance(pr,str): pr = pr.strip()
    if pr: self.h.append(mk_msg(pr))
    if self.tools: kwargs['tools'] = [mk_openai_func(o) for o in self.tools]
    if self.tool_choice: kwargs['tool_choice'] = mk_tool_choice(self.tool_choice)
    res = self.c(self.h, sp=self.sp, stream=stream, **kwargs)
    self.h += mk_toolres(res, ns=self.tools)
    return res

In [ ]:
chat("I'm Jeremy")
chat("What's my name?")

Your name is Jeremy. How can I help you today?

<details>

- id: chatcmpl-AvrnXAdhUOt5M7h2gEJUsUMf50riN
- choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Your name is Jeremy. How can I help you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))]
- created: 1738354859
- model: gpt-4o-2024-08-06
- object: chat.completion
- service_tier: default
- system_fingerprint: fp_4691090a87
- usage: CompletionUsage(completion_tokens=13, prompt_tokens=42, total_tokens=55, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
- _request_id: req_5cbcbd38bb3471e07a7d4f0e27df587c
- __exclude_fields__: {'__exclude_fields__', '_request_id'}

</details>

In [ ]:
chat = Chat(model, sp=sp)
for o in chat("I'm Jeremy", stream=True):
    o = contents(o)
    if o and isinstance(o, str): print(o, end='')

Hello, Jeremy! How can I assist you today?

Check that the o1 reasoning model works and compare 4o default to o1 behavior.

In [ ]:
chat = Chat(model, sp=sp)
chat_o1 = Chat("o1", sp=sp)
problem = "1233 * 4297"
print(f"Correct Answer:\n{problem} = {eval(problem)}")

print("\ngpt-4o Answer:")
r = chat(f"what is {problem}?")
print(contents(r))

print("\no-1 Answer:")
r = chat_o1(f"what is {problem}?")
print(contents(r))

Correct Answer:
1233 * 4297 = 5298201

gpt-4o Answer:
1233 multiplied by 4297 equals 5,295,801.

o-1 Answer:
5,298,201


### Chat tool use

In [ ]:
pr = f"What is {a}+{b}?"
pr

'What is 604542+6458932?'

In [ ]:
chat = Chat(model, sp=sp, tools=[sums])
r = chat(pr)
r

In [ ]:
chat()

## Images

As everyone knows, when testing image APIs you have to use a cute puppy.

In [ ]:
# Image is Cute_dog.jpg from Wikimedia
fn = Path('samples/puppy.jpg')
display.Image(filename=fn, width=200)

In [ ]:
img = fn.read_bytes()

OpenAI expects an image message to have the following structure

```js
{
  "type": "image_url",
  "image_url": {
    "url": f"data:{MEDIA_TYPE};base64,{IMG}"
  }
}
```
`msglm` automatically detects if a message is an image, encodes it, and generates the data structure above.
All we need to do is a create a list containing our image and a query and then pass it to `mk_msg`.

Let's try it out...

In [ ]:
q = "In brief, what color flowers are in this image?"
msg = [mk_msg(img), mk_msg(q)]

In [ ]:
c = Chat(model)
c([img, q])

# Third Party Providers

## Azure OpenAI Service

In [ ]:
#| export
models_azure = ('gpt-4o', 'gpt-4-32k', 'gpt4-1106-preview', 'gpt-35-turbo', 'gpt-35-turbo-16k')

Example Azure usage:
```
azure_endpoint = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-08-01-preview"
)

client = Client(models_azure[0], azure_endpoint)
chat = Chat(cli=client)
chat("I'm Faisal")
```

## Export -

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()